## Cleaning the Yellow Trip data

In [1]:
import pandas as pd

#We will use trip date time with weather date time to merge and clean the dataset
df_trip = pd.read_csv("./data/yellow_tripdata_2024.csv", parse_dates=['tpep_pickup_datetime'])

print(df_trip.columns)

/var/folders/qq/q7zb_56n3nd0hngl2qjjdt2c0000gn/T/ipykernel_92403/442171894.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trip = pd.read_csv("./data/yellow_tripdata_2024.csv", parse_dates=['tpep_pickup_datetime'])


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')


In [2]:
# Drop rows with missing critical fields
df_trip_cleaned = df_trip.dropna(subset=[
    'tpep_pickup_datetime', 'tpep_dropoff_datetime',
    'fare_amount', 'trip_distance', 'passenger_count'
])

# Filter out invalid entries
df_trip_cleaned = df_trip_cleaned[
    (df_trip_cleaned['trip_distance'] > 0) &
    (df_trip_cleaned['fare_amount'] > 0) &
    (df_trip_cleaned['passenger_count'] > 0)
]


#get the date and time from the df
df_trip_cleaned['date'] = df_trip_cleaned['tpep_pickup_datetime'].dt.date
df_trip_cleaned['hour'] = df_trip_cleaned['tpep_pickup_datetime'].dt.hour

#Now we need to aggregate the data by date and hour so that we can match the date and hour with the weather data 
trip_agg = df_trip_cleaned.groupby(['date', 'hour']).agg(
    trip_count=('trip_distance', 'count'),
    avg_distance=('trip_distance', 'mean'),
    avg_fare=('fare_amount', 'mean'),
    avg_passengers=('passenger_count', 'mean')
).reset_index()

# Read it into the DataFrame
trip_agg.to_csv("./data/cleaned_yellow_trip_hourly.csv", index=False)

# Display the first 5 rows of the DataFrame
trip_agg.head(5)

,date,hour,trip_count,avg_distance,avg_fare,avg_passengers
0,2002-12-31,16,1,7.7700,37.30,1.0
1,2002-12-31,22,4,2.5325,14.20,1.0
2,2002-12-31,23,4,7.6875,33.45,1.0
3,2008-12-31,0,1,5.1900,49.90,1.0
4,2008-12-31,22,1,1.6200,11.40,1.0


In [3]:
# Filter data between 2024-01-01 and 2024-12-31

# Convert the column to datetime if it's not already
trip_agg['date'] = pd.to_datetime(trip_agg['date'], errors='coerce')

# Filter rows between Jan 1 and Dec 31, 2024
mask = (trip_agg['date'] >= '2024-01-01') & (trip_agg['date'] <= '2024-12-31')
df_filtered = trip_agg.loc[mask]
(df_filtered['date'].min(), df_filtered['date'].max())

(Timestamp('2024-01-01 00:00:00'), Timestamp('2024-07-01 00:00:00'))

In [4]:
df_filtered.to_csv('./data/cleaned_taxi_trip_hourly.csv', index=False)